In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import datetime

In [ ]:
df_weather=pd.read_csv('/kaggle/input/nyc-flights-2013/nyc_weather.csv')
df_airlines=pd.read_csv('/kaggle/input/nyc-flights-2013/nyc_airlines.csv')
df_flights=pd.read_csv('/kaggle/input/nyc-flights-2013/nyc_flights.csv')
df_planes=pd.read_csv('/kaggle/input/nyc-flights-2013/nyc_planes.csv')
df_airports=pd.read_csv('/kaggle/input/nyc-flights-2013/nyc_airports.csv')

In [ ]:
df_weather

In [ ]:
df_airlines.head()

In [ ]:
df_flights

In [ ]:
df_planes.head()

In [ ]:
df_planes['tailnum'].unique()

In [ ]:
df_airports.head()

In [ ]:
df1=pd.merge(df_flights, df_planes, on='tailnum', how='left')

In [ ]:
df1

In [ ]:
df1=pd.merge(df1,df_weather,on=['time_hour','origin'],how='left')

In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
df1

In [ ]:
df1=df1.drop(['month_y','day_y','hour_y','year'], axis=1)

In [ ]:
df1=df1.rename(columns={'year_x': 'year', 'month_x': 'month','hour_x':'hour','day_x':'day','year_y':'year_plane'})

In [ ]:
df1

In [ ]:
df1.isnull().sum()

In [ ]:
df1=df1.drop(['speed','wind_gust'], axis=1)

In [ ]:
df1=df1.dropna(how='any')

In [ ]:
d_delay = lambda x: 1 if x > 0 else 0

In [ ]:
a_delay = lambda x: 1 if x > 0 else 0

In [ ]:
df1['d_delay']=df1['dep_delay'].apply(d_delay)

In [ ]:
df1['a_delay']=df1['arr_delay'].apply(a_delay)

In [ ]:
df1

In [ ]:
df1.dtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(df1.corr(),annot=True,fmt="1.2f")

In [ ]:
df1['route']=df1['origin'] + '-' + df1['dest']

In [ ]:
df1

In [ ]:
df1['year']=df1['year'].astype(str)
df1['month']=df1['month'].astype(str)
df1['day']=df1['day'].astype(str)

In [ ]:
df1['date']=df1['year']+'-'+df1['month']+'-'+df1['day']

In [ ]:
df1['date']=pd.to_datetime(df1['date'],format='%Y-%m-%d')
df1['date']=df1['date'].dt.tz_localize(None)

In [ ]:
df1_delay=df1[df1['d_delay']==1]

In [ ]:
df1_delay

# 1.Time Series of 'Delay Rate' and 'Average Wind Speed'

In [ ]:
df_flightNO=pd.DataFrame(df1.groupby('date')['flight'].count())
df_flightDL=pd.DataFrame(df1_delay.groupby('date')['flight'].count())

In [ ]:
y1=df_flightNO['flight']
y2=df_flightDL['flight']
y3=y2/y1*100
y4=df1_delay.groupby('date')['wind_speed'].mean()

In [ ]:
fig, ax1 = plt.subplots(figsize=(50,10))
ax2 = ax1.twinx()
ax1.plot(y3,label="delay rate",color='r')
ax2.plot(y4,label="wind speed",color='b')
ax1.legend(loc=1,fontsize=20)
ax2.legend(loc=2,fontsize=20)

plt.show()

# 2. Route analysis

In [ ]:
df1_delay.groupby('route')[('dep_delay','arr_delay')].describe()

# 1) Top 100 routes which have a lot of flights

In [ ]:
df1_delay.groupby('route')['route'].count().sort_values(ascending=False).head(100).plot.bar(figsize=(25,10))

# 2) Top 100 routes which have a lot of delays

In [ ]:
df1_delay.groupby('route')['dep_delay'].count().sort_values(ascending=False).head(100).plot.bar(figsize=(25,10))

# 3) Top 100 routes which have longer average delays

In [ ]:
df1_delay.groupby('route')[('dep_delay')].mean().sort_values(ascending=False).head(100).plot.bar(figsize=(25,10))

# 4) Route LGA-TVC which has the longest average delays

In [ ]:
df1_delay_LT=df1_delay[df1_delay['route']=='LGA-TVC']
sns.barplot(y=df1_delay_LT['date'],x=df1_delay_LT['dep_delay'])

In [ ]:
df1_delay_LT.describe()

In [ ]:
df1.describe()

# By comparing between route LGA-TVC and all data, it seems that 'year_plane' average is older in LGA-TVC

# 5) Multi-factors route analysis-top 200

In [ ]:
df2=df1_delay.sort_values('dep_delay',ascending=False).head(200)

# -By carrier and wind speed

In [ ]:
plt.figure(figsize=(20, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df2['route'],y=df2['dep_delay'],hue=df2['carrier'],size=df2['wind_speed'],data=df2,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# -By carrier and distance

In [ ]:
plt.figure(figsize=(20, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df2['route'],y=df2['dep_delay'],hue=df2['carrier'],size=df2['distance'],data=df2,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# -By carrier and year_plane

In [ ]:
plt.figure(figsize=(20, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df2['route'],y=df2['dep_delay'],hue=df2['carrier'],size=df2['year_plane'],data=df2,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

# 3. Carrier Analysis

# 1) Carriers which have a lot of flights

In [ ]:
df1_delay.groupby('carrier')['carrier'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 2)Carriers  which have a lot of delays

In [ ]:
df1_delay.groupby('carrier')['dep_delay'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 3)Carriers which have longer average delays

In [ ]:
df1_delay.groupby('carrier')['dep_delay'].mean().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 4. Airplane model analysis

# 1) Airplane models which have a  lot of flight

In [ ]:
df1_delay.groupby('model')['model'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 2) Airplane models which have a lot of delays

In [ ]:
df1_delay.groupby('model')['dep_delay'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 3) Airplane models which have longer average delays

In [ ]:
df1_delay.groupby('model')['dep_delay'].mean().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 5. Airplane age analysis

# 1) Airplane age which have a lot of flight

In [ ]:
df1_delay.groupby('year_plane')['year_plane'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 2) Airplane age which have a lot of delays

In [ ]:
df1_delay.groupby('year_plane')['dep_delay'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 3)Airplane age which have longer average delays

In [ ]:
df1_delay.groupby('year_plane')['dep_delay'].mean().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 6. Airport analysis

# 1)Depature numbers

In [ ]:
df1_delay.groupby('origin')['origin'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 2)Depature delays numbers

In [ ]:
df1_delay.groupby('origin')['dep_delay'].count().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 3)Average delays

In [ ]:
df1_delay.groupby('origin')['dep_delay'].mean().sort_values(ascending=False).plot.bar(figsize=(25,10))

# 7. Month analysis

In [ ]:
df1_delay.groupby('month')[('dep_delay')].count().sort_values(ascending=False).plot.bar(figsize=(10,10))